In [1]:
import pandas as pd
import numpy as np
import math
import time

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
vic_data = pd.read_csv('../kennedy/victoria.csv')

In [3]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
FIXED_OPERATIONS_MAINTENANCE = 8.1

TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

In [4]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [5]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    
    df['charge_forecast'] = 0
    df['discharge_forecast'] = 0
    
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    df = df.drop([0], axis=0) 
    
    return df

In [6]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1

    return df

In [7]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1
                
        else:
            thelist = df.iloc[i:][PRICE]
            ser = pd.Series(thelist)

            if (df.at[i,PRICE] <= ser.quantile(CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= ser.quantile(DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1
            
    return df

In [8]:
def calculate(df):
    
    for i in list(df.index):
        price = df.at[i, PRICE]
        
        # update opening capacity
        if (i != 1):
            df.at[i, 'opening'] = df.at[i-1, 'closing']
            
        opening_cap = df.at[i, 'opening']


        # finding raw_power
        if (df.at[i,'charge_forecast'] == 1):
            df.at[i,'raw_power'] = -min(BATTERY_POWER, (BATTERY_CAPACITY-opening_cap)/EFFICIENCY*2)

        if (df.at[i,'discharge_forecast'] == 1):
            df.at[i,'raw_power'] = min(BATTERY_POWER, opening_cap/EFFICIENCY*2)
            
        rawPower = df.at[i, "raw_power"]


        # finding market_dispatch 
        if (rawPower < 0):
            df.at[i,'dispatch'] = (rawPower/2)

        else:
            # EFFICIENCY is already in decimal (0.9) so no need to divide by 100
            df.at[i,'dispatch'] = (rawPower/2 * EFFICIENCY)
            
        dispatch = df.at[i, "dispatch"]


        # finding closing_capacity   
        if (dispatch < 0):
            thecondition = opening_cap - (dispatch * EFFICIENCY)
        else:
            thecondition = opening_cap - (dispatch * (100/(EFFICIENCY*100)))

        df.at[i,'closing'] = round(max(0, min(thecondition, BATTERY_CAPACITY)),0)



        #finding revenue        
        if (dispatch < 0):
            df.at[i,'revenue'] = round(dispatch * price * (1/MARGINAL_LOSS_FACTOR),0)
        else:
            df.at[i,'revenue'] = round(dispatch * price * MARGINAL_LOSS_FACTOR,0)
    
    return df

In [9]:
def run_all(ori_df):
    
    start = time.time()
    
    df2 = create_df(ori_df)
    
    df3 = algorithm3(df2)
    
    df = calculate(df3)
            
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [10]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 121370497
Total days in the dataset: 1322.0
Revenue per day: 91808.2428139183
Time Complexity for running the entire Algorithm 3: 24.360108375549316s


############################################# This is a line breaker #################################################

In [15]:
# Lookahead 5
LOOKAHEAD = 5
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 111988838
Total days in the dataset: 1322.0
Revenue per day: 84711.67776096823
Time Complexity for running the entire Algorithm 3: 21.081890106201172s


In [31]:
# Lookahead 15
LOOKAHEAD = 15
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 113988591
Total days in the dataset: 1322.0
Revenue per day: 86224.3502269289
Time Complexity for running the entire Algorithm 3: 21.221274375915527s


In [23]:
# Change Discharging Percentile
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.70
vic_price = run_all(vic_data)

Total revenue in the dataset: 120502950
Total days in the dataset: 1322.0
Revenue per day: 91152.00453857791
Time Complexity for running the entire Algorithm 3: 23.356554746627808s


120502950

In [16]:
# Change Charging Percentile
# Currently highest Revenue
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.30
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 121879439
Total days in the dataset: 1322.0
Revenue per day: 92193.22163388805
Time Complexity for running the entire Algorithm 3: 22.667230129241943s


In [17]:
# Change Charging and Discharging Percentile
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.30
DISCHARGING_PERCENTILE = 0.70
vic_price = run_all(vic_data)

Total revenue in the dataset: 121236562
Total days in the dataset: 1322.0
Revenue per day: 91706.930408472
Time Complexity for running the entire Algorithm 3: 20.62567710876465s


In [21]:
# Change Charging Percentile
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.40
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 121260584
Total days in the dataset: 1322.0
Revenue per day: 91725.10136157337
Time Complexity for running the entire Algorithm 3: 21.600794553756714s


In [11]:
charge = np.arange(0.2, 0.4, 0.01)
discharge = np.arange(0.6, 0.8, 0.01)
revenue=[]
highest=0
charge_percentile=0
discharge_percentile=0
for i in charge:
    for q in discharge:
        LOOKAHEAD = 10
        CHARGING_PERCENTILE = i
        DISCHARGING_PERCENTILE = q
        vic_price = run_all(vic_data)
        revenue.append(vic_price["revenue"].sum())
        if(vic_price["revenue"].sum()>highest): 
            hightest=vic_price["revenue"].sum()
            charge_percentile=i
            discharge_percentile=q

Total revenue in the dataset: 112388393
Total days in the dataset: 1322.0
Revenue per day: 85013.91301059001
Time Complexity for running the entire Algorithm 3: 24.314722537994385s
Total revenue in the dataset: 112730054
Total days in the dataset: 1322.0
Revenue per day: 85272.35552193646
Time Complexity for running the entire Algorithm 3: 24.691587924957275s
Total revenue in the dataset: 112920663
Total days in the dataset: 1322.0
Revenue per day: 85416.5378214826
Time Complexity for running the entire Algorithm 3: 24.240518808364868s
Total revenue in the dataset: 113154716
Total days in the dataset: 1322.0
Revenue per day: 85593.58245083207
Time Complexity for running the entire Algorithm 3: 24.135218143463135s
Total revenue in the dataset: 115384131
Total days in the dataset: 1322.0
Revenue per day: 87279.9780635401
Time Complexity for running the entire Algorithm 3: 23.833822011947632s
Total revenue in the dataset: 117680038
Total days in the dataset: 1322.0
Revenue per day: 89016.

Total revenue in the dataset: 116205442
Total days in the dataset: 1322.0
Revenue per day: 87901.24205748865
Time Complexity for running the entire Algorithm 3: 24.547780990600586s
Total revenue in the dataset: 118423324
Total days in the dataset: 1322.0
Revenue per day: 89578.91376701967
Time Complexity for running the entire Algorithm 3: 25.94930100440979s
Total revenue in the dataset: 118941393
Total days in the dataset: 1322.0
Revenue per day: 89970.7965204236
Time Complexity for running the entire Algorithm 3: 25.566785097122192s
Total revenue in the dataset: 119126715
Total days in the dataset: 1322.0
Revenue per day: 90110.97957639939
Time Complexity for running the entire Algorithm 3: 25.446061611175537s
Total revenue in the dataset: 119239112
Total days in the dataset: 1322.0
Revenue per day: 90196.0
Time Complexity for running the entire Algorithm 3: 25.690268754959106s
Total revenue in the dataset: 119111504
Total days in the dataset: 1322.0
Revenue per day: 90099.4735249621

Total revenue in the dataset: 119888770
Total days in the dataset: 1322.0
Revenue per day: 90687.42057488654
Time Complexity for running the entire Algorithm 3: 29.671732187271118s
Total revenue in the dataset: 119753005
Total days in the dataset: 1322.0
Revenue per day: 90584.723903177
Time Complexity for running the entire Algorithm 3: 29.86481285095215s
Total revenue in the dataset: 119820379
Total days in the dataset: 1322.0
Revenue per day: 90635.6875945537
Time Complexity for running the entire Algorithm 3: 29.52266001701355s
Total revenue in the dataset: 120512334
Total days in the dataset: 1322.0
Revenue per day: 91159.10287443268
Time Complexity for running the entire Algorithm 3: 29.460605144500732s
Total revenue in the dataset: 120481634
Total days in the dataset: 1322.0
Revenue per day: 91135.88048411498
Time Complexity for running the entire Algorithm 3: 29.490134477615356s
Total revenue in the dataset: 120719765
Total days in the dataset: 1322.0
Revenue per day: 91316.009

Total revenue in the dataset: 121204495
Total days in the dataset: 1322.0
Revenue per day: 91682.67397881996
Time Complexity for running the entire Algorithm 3: 42.888774156570435s
Total revenue in the dataset: 121433139
Total days in the dataset: 1322.0
Revenue per day: 91855.62708018154
Time Complexity for running the entire Algorithm 3: 42.48143672943115s
Total revenue in the dataset: 121501793
Total days in the dataset: 1322.0
Revenue per day: 91907.55900151285
Time Complexity for running the entire Algorithm 3: 42.38670516014099s
Total revenue in the dataset: 121412582
Total days in the dataset: 1322.0
Revenue per day: 91840.0771558245
Time Complexity for running the entire Algorithm 3: 42.73846101760864s
Total revenue in the dataset: 121377451
Total days in the dataset: 1322.0
Revenue per day: 91813.50302571862
Time Complexity for running the entire Algorithm 3: 42.762423515319824s
Total revenue in the dataset: 119732978
Total days in the dataset: 1322.0
Revenue per day: 90569.57

Total revenue in the dataset: 121625701
Total days in the dataset: 1322.0
Revenue per day: 92001.28668683812
Time Complexity for running the entire Algorithm 3: 43.51235866546631s
Total revenue in the dataset: 119970651
Total days in the dataset: 1322.0
Revenue per day: 90749.35779122541
Time Complexity for running the entire Algorithm 3: 43.37632894515991s
Total revenue in the dataset: 119189095
Total days in the dataset: 1322.0
Revenue per day: 90158.1656580938
Time Complexity for running the entire Algorithm 3: 43.75001406669617s
Total revenue in the dataset: 118761805
Total days in the dataset: 1322.0
Revenue per day: 89834.95083207262
Time Complexity for running the entire Algorithm 3: 43.22348928451538s
Total revenue in the dataset: 118279504
Total days in the dataset: 1322.0
Revenue per day: 89470.12405446294
Time Complexity for running the entire Algorithm 3: 44.102670431137085s
Total revenue in the dataset: 115158442
Total days in the dataset: 1322.0
Revenue per day: 87109.260

...

Total revenue in the dataset: 116330436
Total days in the dataset: 1322.0
Revenue per day: 87995.79122541603
Time Complexity for running the entire Algorithm 3: 44.58802103996277s
Total revenue in the dataset: 118363547
Total days in the dataset: 1322.0
Revenue per day: 89533.69667170953
Time Complexity for running the entire Algorithm 3: 44.744107246398926s
Total revenue in the dataset: 120521006
Total days in the dataset: 1322.0
Revenue per day: 91165.66263237518
Time Complexity for running the entire Algorithm 3: 44.73061919212341s
Total revenue in the dataset: 120991183
Total days in the dataset: 1322.0
Revenue per day: 91521.31845688351
Time Complexity for running the entire Algorithm 3: 39.61046242713928s
Total revenue in the dataset: 121078930
Total days in the dataset: 1322.0
Revenue per day: 91587.69288956127
Time Complexity for running the entire Algorithm 3: 37.14413571357727s
Total revenue in the dataset: 121093663
Total days in the dataset: 1322.0
Revenue per day: 91598.83

Total revenue in the dataset: 121192071
Total days in the dataset: 1322.0
Revenue per day: 91673.276096823
Time Complexity for running the entire Algorithm 3: 44.12034559249878s
Total revenue in the dataset: 121086866
Total days in the dataset: 1322.0
Revenue per day: 91593.69591527988
Time Complexity for running the entire Algorithm 3: 44.393630504608154s
Total revenue in the dataset: 121325889
Total days in the dataset: 1322.0
Revenue per day: 91774.5
Time Complexity for running the entire Algorithm 3: 43.64929986000061s
Total revenue in the dataset: 121333497
Total days in the dataset: 1322.0
Revenue per day: 91780.25491679274
Time Complexity for running the entire Algorithm 3: 44.138756275177s
Total revenue in the dataset: 121911325
Total days in the dataset: 1322.0
Revenue per day: 92217.34114977307
Time Complexity for running the entire Algorithm 3: 44.28059935569763s
Total revenue in the dataset: 121817668
Total days in the dataset: 1322.0
Revenue per day: 92146.49621785175
Time

Total revenue in the dataset: 121390616
Total days in the dataset: 1322.0
Revenue per day: 91823.46142208774
Time Complexity for running the entire Algorithm 3: 45.323824882507324s
Total revenue in the dataset: 121253234
Total days in the dataset: 1322.0
Revenue per day: 91719.54160363086
Time Complexity for running the entire Algorithm 3: 45.78806400299072s
Total revenue in the dataset: 121387458
Total days in the dataset: 1322.0
Revenue per day: 91821.0726172466
Time Complexity for running the entire Algorithm 3: 42.36280012130737s
Total revenue in the dataset: 121405267
Total days in the dataset: 1322.0
Revenue per day: 91834.54387291982
Time Complexity for running the entire Algorithm 3: 33.275591135025024s
Total revenue in the dataset: 121252701
Total days in the dataset: 1322.0
Revenue per day: 91719.13842662632
Time Complexity for running the entire Algorithm 3: 32.568806409835815s
Total revenue in the dataset: 121187857
Total days in the dataset: 1322.0
Revenue per day: 91670.0

Total revenue in the dataset: 121347785
Total days in the dataset: 1322.0
Revenue per day: 91791.06278366112
Time Complexity for running the entire Algorithm 3: 28.72699737548828s
Total revenue in the dataset: 121275876
Total days in the dataset: 1322.0
Revenue per day: 91736.6686838124
Time Complexity for running the entire Algorithm 3: 29.279550075531006s
Total revenue in the dataset: 119593699
Total days in the dataset: 1322.0
Revenue per day: 90464.22012102875
Time Complexity for running the entire Algorithm 3: 29.47983145713806s
Total revenue in the dataset: 118756910
Total days in the dataset: 1322.0
Revenue per day: 89831.24810892587
Time Complexity for running the entire Algorithm 3: 30.488908767700195s
Total revenue in the dataset: 118247373
Total days in the dataset: 1322.0
Revenue per day: 89445.81921331317
Time Complexity for running the entire Algorithm 3: 30.734982013702393s
Total revenue in the dataset: 117671777
Total days in the dataset: 1322.0
Revenue per day: 89010.4

0.8000000000000002

In [23]:
revenue.index(np.amax(revenue))

266

In [22]:
#The highest Revenue
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.32
DISCHARGING_PERCENTILE = 0.74
vic_price = run_all(vic_data)

Total revenue in the dataset: 122083673
Total days in the dataset: 1322.0
Revenue per day: 92347.71028744327
Time Complexity for running the entire Algorithm 3: 28.630908489227295s


# DELIVERABLE 3

In [22]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
deliverable3 = vic_data[(vic_data[TIME] >= "2020-07-17 00:00:00") & (vic_data[TIME] < "2020-07-18 00:00:30")]
deliverable3 = deliverable3.reset_index(drop=True)
del3 = run_all(deliverable3)
del3

Total revenue in the dataset: 30139
Total days in the dataset: 1.0
Revenue per day: 30139.0
Time Complexity for running the entire Algorithm 3: 0.019743919372558594s


,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
1,2020-07-17 00:30:00,75.51,0,0,0,0,0,0,1
2,2020-07-17 01:00:00,73.98,0,0,0,0,0,0,0
3,2020-07-17 01:30:00,75.57,0,0,0,0,0,0,1
4,2020-07-17 02:00:00,71.94,0,0,0,0,0,0,0
5,2020-07-17 02:30:00,74.10,0,0,0,0,0,0,0
6,2020-07-17 03:00:00,67.36,0,0,0,0,0,0,0
7,2020-07-17 03:30:00,58.04,-300,-150,-8785,0,135,1,0
8,2020-07-17 04:00:00,51.85,-300,-150,-7848,135,270,1,0
9,2020-07-17 04:30:00,74.53,0,0,0,270,270,0,0
10,2020-07-17 05:00:00,67.32,-300,-150,-10190,270,405,1,0


In [35]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
deliverable3 = vic_data[(vic_data[TIME] >= "2020-07-17 00:00:00") & (vic_data[TIME] < "2020-07-18 00:00:30")]
deliverable3 = deliverable3.reset_index(drop=True)
del3 = run_all(deliverable3)
del3

Total revenue in the dataset: 34360
Total days in the dataset: 1.0
Revenue per day: 34360.0
Time Complexity for running the entire Algorithm 3: 0.026081323623657227s


,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
1,2020-07-17 00:30:00,75.51,0,0,0,0,0,0,1
2,2020-07-17 01:00:00,73.98,0,0,0,0,0,0,0
3,2020-07-17 01:30:00,75.57,0,0,0,0,0,0,1
4,2020-07-17 02:00:00,71.94,0,0,0,0,0,0,0
5,2020-07-17 02:30:00,74.10,0,0,0,0,0,0,0
6,2020-07-17 03:00:00,67.36,0,0,0,0,0,0,0
7,2020-07-17 03:30:00,58.04,-300,-150,-8785,0,135,1,0
8,2020-07-17 04:00:00,51.85,-300,-150,-7848,135,270,1,0
9,2020-07-17 04:30:00,74.53,0,0,0,270,270,0,0
10,2020-07-17 05:00:00,67.32,-300,-150,-10190,270,405,1,0


In [26]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
deliverable3 = vic_data[(vic_data[TIME] >= "2020-07-17 00:00:00") & (vic_data[TIME] < "2020-07-18 00:00:30")]
deliverable3 = deliverable3.reset_index(drop=True)
del3 = run_all(deliverable3)
del3

Total revenue in the dataset: 34362
Total days in the dataset: 1.0
Revenue per day: 34362.0
Time Complexity for running the entire Algorithm 3: 0.06494355201721191s


,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
1,2020-07-17 00:30:00,75.51,0,0,0,0,0,0,1
2,2020-07-17 01:00:00,73.98,0,0,0,0,0,0,1
3,2020-07-17 01:30:00,75.57,0,0,0,0,0,0,1
4,2020-07-17 02:00:00,71.94,0,0,0,0,0,0,0
5,2020-07-17 02:30:00,74.10,0,0,0,0,0,0,1
6,2020-07-17 03:00:00,67.36,0,0,0,0,0,0,0
7,2020-07-17 03:30:00,58.04,-300,-150,-8785,0,135,1,0
8,2020-07-17 04:00:00,51.85,-300,-150,-7848,135,270,1,0
9,2020-07-17 04:30:00,74.53,0,0,0,270,270,0,0
10,2020-07-17 05:00:00,67.32,-300,-150,-10190,270,405,1,0
